In [2]:
#Imports
import os
import xml.etree.ElementTree as ET
import pandas as pd
from datetime import datetime
import pickle
import tqdm
import line_profiler
from operator import itemgetter
from sklearn.preprocessing import OneHotEncoder
import math

In [3]:
def track_id_gen(dist, track):
    return track[0:4]+"-"+str(int(dist))

In [4]:
os.getcwd()
os.chdir(r"C:\Users\Nick\Documents\GitHub\grvmodel\Python\DATA")

In [5]:
#pickle Load results

resultspicklefile = open( 'results-df.npy', 'rb')
resultsdf = pickle.load(resultspicklefile)
resultsdf = pd.DataFrame(resultsdf)
print(resultsdf.head())
split_distances = pd.read_csv("split_distance.csv")
resultsdf["dist"] = resultsdf["dist"].astype(str).str[:-1].astype(float)

resultsdf.run_time = resultsdf.run_time.astype(float)
resultsdf["track_id"] = resultsdf.apply(lambda s: track_id_gen(s["dist"],s["track"]), axis=1)
resultsdf_merged = pd.merge(resultsdf, split_distances, on=["track_id", "track"])

     race_id       date     track  dist        grade  place      dog_id box  \
0  208756700  01 Apr 17  Ballarat  450m  Maiden Heat    1.0  1296100098   2   
1  208756700  01 Apr 17  Ballarat  450m  Maiden Heat    2.0  1177320005   3   
2  208756700  01 Apr 17  Ballarat  450m  Maiden Heat    3.0  1549980026   5   
3  208756700  01 Apr 17  Ballarat  450m  Maiden Heat    4.0  1474670055   7   
4  208756700  01 Apr 17  Ballarat  450m  Maiden Heat    5.0   913850094   8   

  split_times split_margins run_time weigth margin  start_price  \
0        6.73          0.00    26.17   30.2      0         10.6   
1        6.87          0.14    26.18   29.3   0.20          6.1   
2        6.99          0.26    26.26   29.4   1.24         11.9   
3        6.88          0.15    26.36   32.8   2.79          3.3   
4        6.91          0.18    26.42   32.4   3.60         16.0   

         dog_name  
0  SWIFT OPERATOR  
1   LAST PRINCESS  
2  DAINTREE PRINZ  
3    TIC TOC TOBY  
4   RACE ROQUETTE  


In [6]:
betfair_df = pickle.load(open('df-betfairSP.npy', 'rb'))
betfair_df['dateF'] = pd.to_datetime(betfair_df.EVENT_DT, dayfirst=True).dt.date
betfair_df['dog_name'] = betfair_df.dog.str[1:].str.upper()
resultsdf['dateF']= pd.to_datetime(resultsdf.date).dt.date

In [7]:
resultsdf.head()

,race_id,date,track,dist,grade,place,dog_id,box,split_times,split_margins,run_time,weigth,margin,start_price,dog_name,track_id,dateF
0,208756700,01 Apr 17,Ballarat,450.0,Maiden Heat,1.0,1296100098,2,6.73,0.00,26.17,30.2,0,10.6,SWIFT OPERATOR,Ball-450,2017-04-01
1,208756700,01 Apr 17,Ballarat,450.0,Maiden Heat,2.0,1177320005,3,6.87,0.14,26.18,29.3,0.20,6.1,LAST PRINCESS,Ball-450,2017-04-01
2,208756700,01 Apr 17,Ballarat,450.0,Maiden Heat,3.0,1549980026,5,6.99,0.26,26.26,29.4,1.24,11.9,DAINTREE PRINZ,Ball-450,2017-04-01
3,208756700,01 Apr 17,Ballarat,450.0,Maiden Heat,4.0,1474670055,7,6.88,0.15,26.36,32.8,2.79,3.3,TIC TOC TOBY,Ball-450,2017-04-01
4,208756700,01 Apr 17,Ballarat,450.0,Maiden Heat,5.0,913850094,8,6.91,0.18,26.42,32.4,3.60,16.0,RACE ROQUETTE,Ball-450,2017-04-01


In [8]:
tracks = resultsdf_merged.track.unique().tolist()



In [9]:
def input_layer_gen(form):
    race_forms = form.groupby(["race_id"])
    for i,j in race_forms:
        print(i,j)
        break

In [10]:
resultsdf.head(100)

,race_id,date,track,dist,grade,place,dog_id,box,split_times,split_margins,run_time,weigth,margin,start_price,dog_name,track_id,dateF
0,208756700,01 Apr 17,Ballarat,450.0,Maiden Heat,1.0,1296100098,2,6.73,0.00,26.17,30.2,0,10.6,SWIFT OPERATOR,Ball-450,2017-04-01
1,208756700,01 Apr 17,Ballarat,450.0,Maiden Heat,2.0,1177320005,3,6.87,0.14,26.18,29.3,0.20,6.1,LAST PRINCESS,Ball-450,2017-04-01
2,208756700,01 Apr 17,Ballarat,450.0,Maiden Heat,3.0,1549980026,5,6.99,0.26,26.26,29.4,1.24,11.9,DAINTREE PRINZ,Ball-450,2017-04-01
3,208756700,01 Apr 17,Ballarat,450.0,Maiden Heat,4.0,1474670055,7,6.88,0.15,26.36,32.8,2.79,3.3,TIC TOC TOBY,Ball-450,2017-04-01
4,208756700,01 Apr 17,Ballarat,450.0,Maiden Heat,5.0,913850094,8,6.91,0.18,26.42,32.4,3.60,16.0,RACE ROQUETTE,Ball-450,2017-04-01
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,422625068,01 Apr 19,Ballarat,390.0,Maiden Aged Prize Money,1.0,250185375,4,8.91,0.19,23.01,28.5,0,19.8,EXPRESSO LOVE,Ball-390,2019-04-01
96,422625068,01 Apr 19,Ballarat,390.0,Maiden Aged Prize Money,2.0,230053774,6,8.92,0.20,23.05,25.5,0.53,5.6,FELICIA BALE,Ball-390,2019-04-01
97,422625068,01 Apr 19,Ballarat,390.0,Maiden Aged Prize Money,3.0,240988046,1,8.86,0.14,23.13,25.1,1.71,1.6,YANIRA BALE,Ball-390,2019-04-01
98,422625068,01 Apr 19,Ballarat,390.0,Maiden Aged Prize Money,4.0,264507961,2,9.09,0.37,23.15,34.0,1.97,26.5,PYRENEES KNIGHT,Ball-390,2019-04-01


In [12]:
# pickle Load results
resultspicklefile = open("results-df.npy", "rb")
resultsdf = pickle.load(resultspicklefile)
resultsdf = pd.DataFrame(resultsdf)
resultsdf.head()

ohe = OneHotEncoder(dtype=int,sparse=False)
transformed = ohe.fit_transform(resultsdf[['track']])
#enc = OneHotEncoder( categories=tracks)
print(resultsdf.shape)
print(len(transformed))
resultsdf["tracksOnehot"] = transformed.tolist()

split_distances = pd.read_csv("split_distance.csv")
resultsdf["dist"] = resultsdf["dist"].astype(str).str[:-1].astype(float)

resultsdf.run_time = resultsdf.run_time.astype(float)
resultsdf.split_margins = resultsdf.split_margins.astype(float)
resultsdf["track_id"] = resultsdf.apply(
    lambda s: track_id_gen(s["dist"], s["track"]), axis=1
)

resultsdf_merged = pd.merge(resultsdf, split_distances, on=["track_id", "track"])

resultsdf_merged["dateF1"] = pd.to_datetime(resultsdf_merged["date"], format="%d %b %y")
resultsdf_merged["speed"] = pd.to_numeric(resultsdf_merged["run_time"]) / pd.to_numeric(
    resultsdf_merged["dist_x"]
)

resultsdf_merged["split_speed"] = pd.to_numeric(
    resultsdf_merged["split_times"]
) / pd.to_numeric(resultsdf_merged["split_dist_estim"])

resultsdf_merged["box"] = pd.to_numeric(resultsdf_merged["box"])
resultsdf_merged["margin"] = pd.to_numeric(resultsdf_merged["margin"])
resultsdf_merged["weigth"] = pd.to_numeric(resultsdf_merged["weigth"])
resultsdf_merged["dateF"] = pd.to_datetime(resultsdf_merged["date"], format="%d %b %y").dt.date

resultsdf_merged = pd.merge(
    resultsdf_merged, betfair_df, how="left", on=["dateF", "dog_name"]
)

form = resultsdf_merged.sort_values("dateF", ascending=False)

with open("results-df-merged-prices.npy", "wb") as fp:   #Pickling
    
    pickle.dump(form, fp)

    
print(form.head())


race_forms = form.groupby(["race_id"])
dog_forms = form.groupby(["dog_id"])
print(race_forms)
race_input_layer = []
race_classes = []
full_details = []
dog_stats_df = []
resultsdf_merged.head(50)

for i, j in tqdm.tqdm(race_forms):
    try:
        race_date = pd.to_datetime(j["dateF"].iloc[0])
        # print(f"{race_date=}")
        dogs = j.dog_id
        # prev_form = form[(form["date"]< race_date)]
        race_id = i
        # print(prev_form.shape)
        race_dist = j.dist_x.iloc[1]
        trackOHE = j.tracksOnehot.iloc[1]
        dog_info = []

        # win_price = j[j.place == 1].start_price.iloc[0]

        # if win_price < 2:
        #     win_class = [1, 0, 0, 0]
        # if 2 < win_price < 5:
        #     win_class = [0, 1, 0, 0]

        i = 0
        for dog in dogs:
            # print(f"dog id = {dog}")
            prev_form = dog_forms.get_group(dog)
            prev_form = prev_form[(prev_form["dateF"] < race_date)]
            prev_races = prev_form.shape[0]
            dog_speed_avg = prev_form.speed.mean()
            dog_speed_sd = prev_form.speed.std()
            dog_speed_max = prev_form.speed.max()
            dog_split_avg = prev_form.split_speed.mean()
            dog_split_sd = prev_form.split_speed.std()
            dog_split_max = prev_form.split_speed.max()
            dog_split_mg_avg = prev_form.split_margins.mean()
            dog_margin_mean = prev_form.margin.mean()
            dog_margin_sd = prev_form.margin.std()
            if prev_races:
                dog_margin_last = prev_form.margin.iloc[0]
                dog_speed_last = prev_form.speed.iloc[0]
            else:
                dog_margin_last = -1
                dog_speed_last = -1
            # print(dog_margin_last)
            dog_race_count = prev_form.shape[0]
            dog_wins = prev_form[prev_form["place"] == 1].shape[0]
            dog_places = prev_form[prev_form["place"] <= 3].shape[0]
            dog_box = j.box.iloc[i]
            dog_weight = j.weigth.iloc[i] # weight not in forms for prediction going to use last weight
            dog_stats = (
                dog,
                [
                    dog_box,
                    dog_speed_avg,
                    dog_speed_sd,
                    dog_speed_max,
                    dog_split_avg,
                    dog_split_sd,
                    dog_split_max,
                    dog_split_mg_avg,
                    dog_margin_mean,
                    dog_margin_sd,
                    dog_race_count,
                    dog_wins,
                    dog_places,
                    dog_weight,
                    dog_speed_last,
                    dog_margin_last
                ],
                j.place.iloc[i],
                j.start_price.iloc[i],
                j.margin.iloc[i],
                j.BSP.iloc[i],
            )
            stats = [
                    dog_box,
                    dog_speed_avg,
                    dog_speed_sd,
                    dog_speed_max,
                    dog_split_avg,
                    dog_split_sd,
                    dog_split_max,
                    dog_split_mg_avg,
                    dog_margin_mean,
                    dog_margin_sd,
                    dog_race_count,
                    dog_wins,
                    dog_places,
                    dog_weight,
                    dog_speed_last,
                    dog_margin_last
                    ]
            stats = [-1 if math.isnan(x) else x for x in stats]
            dog_info.append(dog_stats)
            dog_stats_df.append([dog,
                    race_id,
                    race_date,
                    trackOHE,
                    race_dist,
                    stats,
                    j.place.iloc[i],
                    j.start_price.iloc[i],
                    j.margin.iloc[i],
                    j.BSP.iloc[i]])
            #print(f"{dog_stats_df=}")
            i = i + 1


        dog_info.sort(key=itemgetter(0))
        #dog_stats_df.append(dog_info)
        # input_layer = (
        #     trackOHE
        #     + [race_dist]
        #     + [item for sublist in [x[1] for x in dog_info] for item in sublist]
        # )
        
        # # if len(dogs) > 8:
        # #     print(f"{race_id=}")
        # #     print(f"{race_date=}")
        # #     print(f"{j.track_x.iloc[1]=}")
        # classes = [x[2] for x in dog_info]
        # prices = [x[3] for x in dog_info]
        # margins = [x[4] for x in dog_info]
        # betfairSP = [x[5] for x in dog_info]
        # race_input_layer.append(input_layer)
        # race_classes.append(classes)
        # full_details.append(
        #     (race_id, input_layer, classes, prices,  margins, betfairSP)
        # )
    except Exception as e:
        print(f"{e=}")

(532770, 15)
532770
          race_id       date          track  dist_x        grade  place  \
395393  823572866  07 Sep 22           Sale   440.0   Grade 5 T3    1.0   
276674  823708134  07 Sep 22  Meadows (MEP)   525.0  Maiden Heat    2.0   
395404  823572867  07 Sep 22           Sale   440.0   Grade 5 T3    5.0   
395405  823572867  07 Sep 22           Sale   440.0   Grade 5 T3    6.0   
395406  823572867  07 Sep 22           Sale   440.0   Grade 5 T3    7.0   

           dog_id  box split_times  split_margins  ...  PPMAX  PPMIN  IPMAX  \
395393  447244247    2        5.49           0.15  ...    NaN    NaN    NaN   
276674  632136255    4        5.22           0.05  ...    NaN    NaN    NaN   
395404  424312450    4        5.51           0.23  ...    NaN    NaN    NaN   
395405  444701628    3        5.48           0.20  ...    NaN    NaN    NaN   
395406  465422497    5        5.52           0.24  ...    NaN    NaN    NaN   

        IPMIN MORNINGTRADEDVOL PPTRADEDVOL IPTRADEDVOL

 68%|██████▊   | 49037/72075 [14:49<05:41, 67.52it/s]

e=IndexError('single positional indexer is out-of-bounds')


 82%|████████▏ | 59295/72075 [17:43<03:37, 58.71it/s]

e=IndexError('single positional indexer is out-of-bounds')


100%|██████████| 72075/72075 [21:18<00:00, 56.39it/s]


full_details

In [ ]:
asdf

testdf = pd.DataFrame(race_input_layer)
print(testdf.shape)
no_nans = testdf.dropna()
print(no_nans.shape)

NameError: name 'asdf' is not defined

In [ ]:
with open("total_list.npy", "wb") as fp:   #Pickling
    
    pickle.dump(full_details, fp)

In [ ]:
[item for sublist in [x[1] for x in dog_stats_df] for item in sublist]

In [ ]:
dog_stats_df["stats"]

TypeError: list indices must be integers or slices, not str

In [13]:
dfx = pd.DataFrame( dog_stats_df, columns=["dogid", "raceid", "date", "trackOHE","dist", "stats", "place", "startprice", "margin", "bfSP"]) 
z =dfx.stats
z[1]

[6,
 0.061256388082696595,
 0.0011419304507069529,
 0.06268354430379747,
 0.06081872302453238,
 0.0011619344086033774,
 0.062032575589083185,
 0.2588888888888889,
 14.535555555555554,
 6.325172154003224,
 9,
 0,
 0,
 26.6,
 0.06093333333333333,
 3.93]

In [15]:
z

0         [1, 0.059672102206736355, 0.001303845494665117...
1         [6, 0.061256388082696595, 0.001141930450706952...
2         [3, 0.05918524613560071, 0.0009662708972431771...
3         [7, 0.05796197139049795, 0.0010752646024627035...
4         [2, 0.05945284327323162, 0.0007404203490765129...
                                ...                        
530017    [2, 0.057828082170100215, 0.000988424427264397...
530018    [5, 0.057832376285840065, 0.000857194927614034...
530019    [8, 0.057757868814331315, 0.000860978290429466...
530020    [7, 0.058740114629819665, 0.000743159508124979...
530021    [4, 0.05809288162817575, 0.000805936031533935,...
Name: stats, Length: 530022, dtype: object

In [14]:
with open("dog_stats_df.npy", "wb") as fp:   #Pickling
    
    pickle.dump(dfx, fp)

In [ ]:
file = open( "total_list.npy", 'rb')
data = pickle.load(file)
raceIDs, inputs, classes, prices, win_price, margins, betfairSP = zip(*data)
betfairSP

([120.0, 12.0, 440.0, 5.599895556, 5.8, 3.0, nan, 50.0],
 [21.197895273503093,
  1.8815589025241928,
  23.296934,
  6.4,
  6.4,
  56.41416643969713,
  20.262803679636438,
  20.4984229903043],
 [7.4,
  91.03969616041284,
  30.45138480166854,
  1.7995775616815202,
  6.733082163684885,
  20.0,
  14.464201088211992,
  26.508447421144847],
 [nan,
  18.0,
  45.76047081084921,
  nan,
  4.832985005122609,
  3.0359346341097995,
  25.787295672437185,
  7.2620745492959395],
 [3.4,
  16.969517280344277,
  11.408280089,
  32.0,
  2.8291454008706207,
  9.931053784615065,
  27.0,
  26.0],
 [9.2,
  13.705378454,
  15.285044260318394,
  3.8718340105350224,
  12.5,
  15.21603800763725,
  6.085789691736865,
  3.942716832],
 [4.88325705054638,
  27.660661303531377,
  5.699330910589779,
  16.5,
  13.414623930350643,
  14.887840753,
  4.222530257670879,
  8.2],
 [18.0,
  nan,
  nan,
  4.550833096578717,
  8.742283643744887,
  nan,
  38.84057459088042],
 [240.97036823254075,
  nan,
  3.2798251233738847,
  14

In [ ]:
bfSPdf = pd.DataFrame(betfairSP)
bfSPdf

,0,1,2,3,4,5,6,7
0,120.000000,12.000000,440.000000,5.599896,5.800000,3.000000,NaN,50.000000
1,21.197895,1.881559,23.296934,6.400000,6.400000,56.414166,20.262804,20.498423
2,7.400000,91.039696,30.451385,1.799578,6.733082,20.000000,14.464201,26.508447
3,NaN,18.000000,45.760471,NaN,4.832985,3.035935,25.787296,7.262075
4,3.400000,16.969517,11.408280,32.000000,2.829145,9.931054,27.000000,26.000000
...,...,...,...,...,...,...,...,...
71763,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
71764,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
71765,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
71766,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
with open("input_list.npy", "wb") as fp:   #Pickling
    
    pickle.dump(race_input_layer, fp)

with open("classes_list.npy", "wb") as fp:   #Pickling
    
    pickle.dump(race_classes, fp)

In [ ]:
inputlayer = open( "input_list.npy", 'rb')
inputlayerdf = pickle.load(inputlayer)
inputlayerdf

[[0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  1,
  0,
  0,
  525.0,
  1,
  0.059672102206736355,
  0.0013038454946651171,
  0.06148571428571429,
  0.05890175351276232,
  0.0009527063639297911,
  0.059994277847454834,
  0.17400000000000002,
  8.902000000000001,
  5.868779259777964,
  5,
  0,
  2,
  '26.8',
  8,
  0.05848506493506494,
  0.0008907936012381001,
  0.06027272727272727,
  0.058515294245794845,
  0.0009500584460310687,
  0.06052134268110914,
  0.09090909090909091,
  5.381818181818182,
  5.208435116581214,
  11,
  1,
  7,
  '26.9',
  6,
  0.061256388082696595,
  0.0011419304507069529,
  0.06268354430379747,
  0.06081872302453238,
  0.0011619344086033771,
  0.062032575589083185,
  0.2588888888888889,
  14.535555555555554,
  6.325172154003224,
  9,
  0,
  0,
  '26.6',
  5,
  0.05816878881987578,
  0.0013401291879889708,
  0.059391304347826086,
  0.05788880463785036,
  0.00204291954498698,
  0.06056436558833152,
  0.065,
  5.805000000000001,
  4.601206363552932,
  